In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df = pd.read_csv('sepsis_comorbidity.csv')

In [ ]:
df_vitals = pd.read_csv('vitals_long.csv')

In [ ]:
# data = data.sort_values(["stay_id","hour"]).reset_index(drop=True)

# future = data[["stay_id","hour","sepsis3_bin"]].copy()

# f1 = future.rename(columns={"sepsis3_bin":"s_h1"}); f1["hour"] = f1["hour"] - 1
# f2 = future.rename(columns={"sepsis3_bin":"s_h2"}); f2["hour"] = f2["hour"] - 2
# f3 = future.rename(columns={"sepsis3_bin":"s_h3"}); f3["hour"] = f3["hour"] - 3
# f4 = future.rename(columns={"sepsis3_bin":"s_h4"}); f4["hour"] = f4["hour"] - 4

# lab = data.merge(f1, on=["stay_id","hour"], how="left") \
#           .merge(f2, on=["stay_id","hour"], how="left") \
#           .merge(f3, on=["stay_id","hour"], how="left") \
#           .merge(f4, on=["stay_id","hour"], how="left")

# for c in ["s_h1","s_h2","s_h3","s_h4"]:
#     lab[c] = lab[c].fillna(0).astype(int)

# future_any = (lab["s_h1"] | lab["s_h2"] | lab["s_h3"] | lab["s_h4"]).astype(int)

# lab["y_next_1to4h"] = ((lab["sepsis3_bin"] == 0) & (future_any == 1)).astype(int)

# # Drop leakage rows where already septic at time t
# data_labeled = lab[lab["sepsis3_bin"] == 0].copy()

# print("y_next_1to4h distribution:\n", data_labeled["y_next_1to4h"].value_counts(dropna=False))
# print("data_labeled shape:", data_labeled.shape)


In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import GroupShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, average_precision_score

# ---------------------------
# Helpers
# ---------------------------
def sepsis_to_01(x: pd.Series) -> pd.Series:
    """Robust conversion to 0/1 from bool, strings, ints."""
    s = x.copy()
    if pd.api.types.is_bool_dtype(s):
        return s.astype(int)
    ss = s.astype(str).str.strip().str.lower()
    mapping = {"true":1, "false":0, "1":1, "0":0, "yes":1, "no":0, "t":1, "f":0}
    out = ss.map(mapping)
    out_num = pd.to_numeric(s, errors="coerce")
    out = out.fillna(out_num)
    return out.fillna(0).astype(int)

def split_stays_with_pos(pos_stays, neg_stays, test_size=0.2, seed=42):
    """Ensure both train and test contain positive stays if any exist."""
    rng = np.random.default_rng(seed)
    pos_stays = np.array(list(pos_stays))
    neg_stays = np.array(list(neg_stays))
    rng.shuffle(pos_stays); rng.shuffle(neg_stays)

    if len(pos_stays) == 0:
        # no positives available
        return set(neg_stays), set()

    n_pos_test = max(1, int(len(pos_stays) * test_size))
    n_neg_test = int(len(neg_stays) * test_size)

    test_stays = np.concatenate([pos_stays[:n_pos_test], neg_stays[:n_neg_test]])
    train_stays = np.concatenate([pos_stays[n_pos_test:], neg_stays[n_neg_test:]])
    return set(train_stays), set(test_stays)

# ---------------------------
# 0) Checks
# ---------------------------
required_vitals_cols = {"subject_id","hadm_id","stay_id","hour","concept","valuenum","intime"}
missing_v = required_vitals_cols - set(df_vitals.columns)
if missing_v:
    raise ValueError(f"df_vitals missing columns: {missing_v}")

required_df_cols = {"subject_id","hadm_id","stay_id","sepsis3"}
missing_d = required_df_cols - set(df.columns)
if missing_d:
    raise ValueError(f"df missing columns: {missing_d}")

KEYS = ["subject_id","hadm_id","stay_id","hour"]

# ---------------------------
# 1) Clean vitals + build hourly skeleton (ALL stays)
# ---------------------------
df_v = df_vitals.copy()
df_v["intime"] = pd.to_datetime(df_v["intime"], errors="coerce")
df_v["charttime"] = pd.to_datetime(df_v["charttime"], errors="coerce") if "charttime" in df_v.columns else pd.NaT
df_v["hour"] = pd.to_numeric(df_v["hour"], errors="coerce")
df_v = df_v.dropna(subset=["stay_id","hour","concept","valuenum","intime"]).copy()
df_v["hour"] = df_v["hour"].astype(int)

skeleton = (
    df_v.groupby(KEYS, as_index=False)
    .size()
    .drop(columns=["size"])
)

# ---------------------------
# 2) Wide vitals features
# ---------------------------
v_agg = (
    df_v.groupby(KEYS + ["concept"])["valuenum"]
      .agg(["mean","min","max","std","count","median"])
      .reset_index()
)

v_wide = v_agg.pivot_table(
    index=KEYS,
    columns="concept",
    values=["mean","min","max","std","count","median"]
)
v_wide.columns = [f"{concept}__{stat}" for stat, concept in v_wide.columns]
v_wide = v_wide.reset_index()

# ---------------------------
# 3) Merge df onto skeleton (stay-level info broadcast to all hours)
# ---------------------------
d = df.copy()
for c in ["sofa_time","suspected_infection_time","admittime","dischtime","antibiotic_time","culture_time"]:
    if c in d.columns:
        d[c] = pd.to_datetime(d[c], errors="coerce")

# stay-level merge (most robust; avoids your earlier 'hour' KeyError)
base = skeleton.merge(d, on=["subject_id","hadm_id","stay_id"], how="left")

# merge vitals features
data = base.merge(v_wide, on=KEYS, how="left")

# Add intime per stay from vitals to define timestamps
stay_intime = df_v.groupby("stay_id")["intime"].min()
data["intime"] = data["stay_id"].map(stay_intime)
data["intime"] = pd.to_datetime(data["intime"], errors="coerce")

# ---------------------------
# 4) Build label from onset timestamp (THIS FIXES THE "all zeros" issue)
# ---------------------------
data["sepsis3_bin"] = sepsis_to_01(data["sepsis3"])

# pick onset column
if "sofa_time" in data.columns and data["sofa_time"].notna().any():
    onset_col = "sofa_time"
elif "suspected_infection_time" in data.columns and data["suspected_infection_time"].notna().any():
    onset_col = "suspected_infection_time"
else:
    raise ValueError(
        "No usable onset time column found. Need non-null sofa_time or suspected_infection_time "
        "to label 'onset in next 1–4 hours'."
    )

data["onset_time"] = pd.to_datetime(data[onset_col], errors="coerce")

# current time at each hour
data["t0"] = data["intime"] + pd.to_timedelta(data["hour"], unit="h")

# label: onset in (t0, t0+4h]
data["y_next_1to4h"] = (
    (data["sepsis3_bin"] == 1) &
    (data["onset_time"].notna()) &
    (data["onset_time"] > data["t0"]) &
    (data["onset_time"] <= data["t0"] + pd.Timedelta(hours=4))
).astype(int)

# OPTIONAL: drop rows at/after onset to avoid training on post-event info
data_labeled = data[(data["onset_time"].isna()) | (data["t0"] < data["onset_time"])].copy()

print("Using onset column:", onset_col)
print("Label distribution:", data_labeled["y_next_1to4h"].value_counts(dropna=False).to_dict())

# sanity: positive stays count
stay_label = data_labeled.groupby("stay_id")["y_next_1to4h"].max()
pos_stays = set(stay_label[stay_label == 1].index)
neg_stays = set(stay_label[stay_label == 0].index)
print("Num positive stays:", len(pos_stays))
print("Num negative stays:", len(neg_stays))

if len(pos_stays) == 0:
    raise ValueError(
        "Still 0 positive stays after labeling. That means onset_time never falls within 1–4h of any hour row.\n"
        "Check that your 'hour' is aligned to ICU intime and that onset_time is truly within the ICU window."
    )

# ---------------------------
# 5) Build features (drop IDs/times/labels) + train/test split by stay_id
# ---------------------------
drop_cols = set(KEYS + [
    "sepsis3","sepsis3_bin","y_next_1to4h",
    "onset_time","t0","intime",
    "suspected_infection_time","sofa_time",
    "admittime","dischtime","antibiotic_time","culture_time"
])
feature_cols = [c for c in data_labeled.columns if c not in drop_cols]

X = data_labeled[feature_cols]
y = data_labeled["y_next_1to4h"].astype(int)
groups = data_labeled["stay_id"]

# split stays ensuring positives in test
train_stays, test_stays = split_stays_with_pos(pos_stays, neg_stays, test_size=0.2, seed=42)
train_mask = data_labeled["stay_id"].isin(train_stays)
test_mask  = data_labeled["stay_id"].isin(test_stays)

X_train, X_test = X.loc[train_mask].copy(), X.loc[test_mask].copy()
y_train, y_test = y.loc[train_mask].copy(), y.loc[test_mask].copy()

print("y_train:", y_train.value_counts().to_dict(), " | y_test:", y_test.value_counts().to_dict())

# Drop all-NaN columns in train (fixes SimpleImputer warning)
all_nan_cols = X_train.columns[X_train.isna().all()].tolist()
if all_nan_cols:
    print("Dropping all-NaN train cols:", all_nan_cols)
    X_train.drop(columns=all_nan_cols, inplace=True)
    X_test.drop(columns=all_nan_cols, inplace=True)

# ---------------------------
# 6) Train RandomForest + probabilities
# ---------------------------
model = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("rf", RandomForestClassifier(
        n_estimators=500,
        max_depth=None,
        min_samples_leaf=5,
        n_jobs=-1,
        random_state=42,
        class_weight="balanced_subsample"
    ))
])

model.fit(X_train, y_train)

proba = model.predict_proba(X_test)
if proba.shape[1] < 2:
    raise RuntimeError("Model trained on one class unexpectedly. Check y_train distribution above.")
proba_test = proba[:, 1]

# metrics (only if y_test has both classes)
if y_test.nunique() == 2:
    print("ROC-AUC:", roc_auc_score(y_test, proba_test))
    print("PR-AUC:", average_precision_score(y_test, proba_test))
else:
    print("⚠️ y_test has one class; AUC metrics not defined. y_test:", y_test.value_counts().to_dict())

data_labeled.loc[X_test.index, "pred_proba_next_1to4h"] = proba_test

# show sample
display_cols = ["stay_id","hour","y_next_1to4h","pred_proba_next_1to4h", onset_col]
print(data_labeled[display_cols].head(30))


In [ ]:

proba_all = model.predict_proba(X)   # X is full feature matrix on data_labeled
if proba_all.shape[1] < 2:
    raise RuntimeError("Model trained on one class unexpectedly. Check y_train distribution above.")
data_labeled["pred_proba_next_1to4h"] = proba_all[:, 1]

proba_test = data_labeled.loc[X_test.index, "pred_proba_next_1to4h"].values

if y_test.nunique() == 2:
    print("ROC-AUC:", roc_auc_score(y_test, proba_test))
    print("PR-AUC:", average_precision_score(y_test, proba_test))
else:
    print("⚠️ y_test has one class; AUC metrics not defined. y_test:", y_test.value_counts().to_dict())

display_cols = ["stay_id","hour","y_next_1to4h","pred_proba_next_1to4h", onset_col]
print(data_labeled[display_cols].head(30))


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import (
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
    accuracy_score,
    balanced_accuracy_score,
    roc_auc_score,
    average_precision_score
)


y_true = data_labeled.loc[X_test.index, "y_next_1to4h"].astype(int).values
p_hat  = data_labeled.loc[X_test.index, "pred_proba_next_1to4h"].astype(float).values

def eval_at_threshold(y_true, p_hat, thr=0.5):
    y_pred = (p_hat >= thr).astype(int)

    cm = confusion_matrix(y_true, y_pred, labels=[0,1])
    tn, fp, fn, tp = cm.ravel()

    # Core
    sensitivity_recall = tp / (tp + fn) if (tp + fn) else np.nan   # TPR / Recall
    specificity        = tn / (tn + fp) if (tn + fp) else np.nan   # TNR
    precision          = tp / (tp + fp) if (tp + fp) else np.nan   # PPV
    npv                = tn / (tn + fn) if (tn + fn) else np.nan

    f1 = f1_score(y_true, y_pred, zero_division=0)
    acc = accuracy_score(y_true, y_pred)
    bal_acc = balanced_accuracy_score(y_true, y_pred)

    # "Stuff" people often want
    fpr = fp / (fp + tn) if (fp + tn) else np.nan                 # 1 - specificity
    fnr = fn / (fn + tp) if (fn + tp) else np.nan                 # 1 - recall
    tpr = sensitivity_recall

    return {
        "threshold": thr,
        "TP": tp, "FP": fp, "TN": tn, "FN": fn,
        "accuracy": acc,
        "balanced_accuracy": bal_acc,
        "precision (PPV)": precision,
        "recall/sensitivity (TPR)": sensitivity_recall,
        "specificity (TNR)": specificity,
        "NPV": npv,
        "F1": f1,
        "FPR": fpr,
        "FNR": fnr
    }

# ---- Single threshold (default 0.5) ----
metrics_05 = eval_at_threshold(y_true, p_hat, thr=0.5)
print(pd.Series(metrics_05))

# ---- Also report threshold-free metrics ----
if len(np.unique(y_true)) == 2:
    print("\nThreshold-free metrics:")
    print("ROC-AUC:", roc_auc_score(y_true, p_hat))
    print("PR-AUC:", average_precision_score(y_true, p_hat))
else:
    print("\n⚠️ y_true has one class in this test split; AUC metrics not defined.")

# ---- Sweep thresholds to pick a good operating point ----
thresholds = np.linspace(0.01, 0.99, 99)
rows = [eval_at_threshold(y_true, p_hat, thr=t) for t in thresholds]
df_thr = pd.DataFrame(rows)

# Example: best F1 threshold
best_f1_row = df_thr.iloc[df_thr["F1"].idxmax()]
print("\nBest F1 operating point:")
print(best_f1_row[["threshold","F1","precision (PPV)","recall/sensitivity (TPR)","specificity (TNR)","TP","FP","TN","FN"]])

# Example: threshold with recall >= 0.80 and max specificity
target_recall = 0.80
eligible = df_thr[df_thr["recall/sensitivity (TPR)"] >= target_recall].copy()
if len(eligible) > 0:
    best_spec = eligible.iloc[eligible["specificity (TNR)"].idxmax()]
    print(f"\nBest specificity with recall >= {target_recall}:")
    print(best_spec[["threshold","specificity (TNR)","recall/sensitivity (TPR)","precision (PPV)","F1","TP","FP","TN","FN"]])
else:
    print(f"\nNo threshold achieved recall >= {target_recall}. Try lowering target_recall.")

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import (
    confusion_matrix, roc_auc_score, average_precision_score
)


y_true = data_labeled.loc[X_test.index, "y_next_1to4h"].astype(int).values
p_hat  = data_labeled.loc[X_test.index, "pred_proba_next_1to4h"].astype(float).values

def eval_at_threshold(y_true, p_hat, thr):
    y_pred = (p_hat >= thr).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()

    sensitivity = tp / (tp + fn) if (tp + fn) else np.nan   # recall / TPR
    specificity = tn / (tn + fp) if (tn + fp) else np.nan   # TNR
    precision   = tp / (tp + fp) if (tp + fp) else np.nan   # PPV
    npv         = tn / (tn + fn) if (tn + fn) else np.nan

    acc = (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) else np.nan
    bal_acc = (sensitivity + specificity) / 2 if (np.isfinite(sensitivity) and np.isfinite(specificity)) else np.nan
    f1 = (2 * precision * sensitivity / (precision + sensitivity)) if (precision + sensitivity) else np.nan

    fpr = fp / (fp + tn) if (fp + tn) else np.nan
    fnr = fn / (fn + tp) if (fn + tp) else np.nan

    return {
        "threshold": thr,
        "TP": tp, "FP": fp, "TN": tn, "FN": fn,
        "accuracy": acc,
        "balanced_acc": bal_acc,
        "precision": precision,
        "recall/sensitivity": sensitivity,
        "specificity": specificity,
        "NPV": npv,
        "F1": f1,
        "FPR": fpr,
        "FNR": fnr
    }

summary = {}
if len(np.unique(y_true)) == 2:
    summary["ROC-AUC"] = roc_auc_score(y_true, p_hat)
    summary["PR-AUC"]  = average_precision_score(y_true, p_hat)
else:
    summary["ROC-AUC"] = np.nan
    summary["PR-AUC"]  = np.nan

summary_df = pd.DataFrame([summary])
print("=== Threshold-free summary (test set) ===")
display(summary_df.style.format("{:.4f}"))

# common thresholds
threshold_list = [0.1, 0.2, 0.3, 0.5]
rows = [eval_at_threshold(y_true, p_hat, t) for t in threshold_list]
metrics_df = pd.DataFrame(rows)

print("=== Metrics at selected thresholds (test set) ===")
display(
    metrics_df[[
        "threshold","TP","FP","TN","FN",
        "precision","recall/sensitivity","specificity","F1",
        "accuracy","balanced_acc","NPV","FPR","FNR"
    ]].style
      .format({
          "precision":"{:.3f}",
          "recall/sensitivity":"{:.3f}",
          "specificity":"{:.3f}",
          "F1":"{:.3f}",
          "accuracy":"{:.3f}",
          "balanced_acc":"{:.3f}",
          "NPV":"{:.3f}",
          "FPR":"{:.3f}",
          "FNR":"{:.3f}",
      })
)

thresholds = np.linspace(0.01, 0.99, 99)
sweep = pd.DataFrame([eval_at_threshold(y_true, p_hat, t) for t in thresholds])

# Best F1 threshold
best_f1 = sweep.iloc[sweep["F1"].idxmax()][[
    "threshold","F1","precision","recall/sensitivity","specificity","TP","FP","TN","FN"
]]

# Best specificity with recall >= target
target_recall = 0.80
eligible = sweep[sweep["recall/sensitivity"] >= target_recall]
best_spec = None
if len(eligible) > 0:
    best_spec = eligible.iloc[eligible["specificity"].idxmax()][[
        "threshold","specificity","recall/sensitivity","precision","F1","TP","FP","TN","FN"
    ]]

print("=== Best operating points (test set) ===")
best_tbl = pd.DataFrame([best_f1])
best_tbl["criterion"] = "Best F1"
best_tbl = best_tbl[["criterion"] + [c for c in best_tbl.columns if c != "criterion"]]

if best_spec is not None:
    tmp = pd.DataFrame([best_spec])
    tmp["criterion"] = f"Max specificity with recall ≥ {target_recall}"
    tmp = tmp[["criterion"] + [c for c in tmp.columns if c != "criterion"]]
    best_tbl = pd.concat([best_tbl, tmp], ignore_index=True)

display(
    best_tbl.style.format({
        "threshold":"{:.2f}",
        "F1":"{:.3f}",
        "precision":"{:.3f}",
        "recall/sensitivity":"{:.3f}",
        "specificity":"{:.3f}",
    })
)
